In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)

# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [155]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    unique_id: str = db_transform.variables.unique_id # Column name to use as ID for ic_{id}_ptl modification
    trans_id: str = db_transform.variables.trans_id # Column name to use for Transaction ID
    brand_col: str = db_transform.variables.brand_col # Column name to use for Brand
    patient_id: str = db_transform.variables.patient_id # Column name to use for Patient ID
    pharmacy: str = db_transform.variables.pharmacy # Column name to use for Pharmacy
    status_date: str = db_transform.variables.status_date # Column name to use for Status Date
    referral_date: str = db_transform.variables.referral_date # Column name to use for Referral Date
    status: str = db_transform.variables.status # Column name to use for Status
    substatus: str = db_transform.variables.substatus # Column name to use for Substatus
    hierarchy: str = db_transform.variables.hierarchy # Column name to use for Hierarchy
    pending_status_code: str = db_transform.variables.pending_status_code # Pending status code (customer-specific)
    cancel_status_code: str = db_transform.variables.cancel_status_code # Cancel status code (customer-specific)
    pending_too_long_code: str = db_transform.variables.pending_too_long_code # Pending Too Long status code
    ptl_threshold: str = db_transform.variables.ptl_threshold # Number of days threshold for inserting a PENDING TOO LONG record (customer-specific)


In [ ]:
transform = Transform()

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Pending Too Long enrichment. Creates a 'Cancelled - Pending Too Long' record in cases where last status reported is PENDING, and no followup has been received in at least 60 days.  This is used as part of the Fill Rate enrichment. See logic diagram below:

<img src = 'assets/pending_too_long.svg' style="width:600px;">

### Transformation

In [ ]:
### Retrieve current dataset from contract
from core.dataset_diff import DatasetDiff

diff = DatasetDiff(db_transform.id)
df = diff.get_diff(transform_name=transform.input_transform, values=[run_id])

In [ ]:
df.shape

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

### DATA CLEANING: ADDRESS THIS SECTION BEFORE PIPELINE INTEGRATION

### APPLY TRANSFORM LOGIC

In [ ]:
# Assign Patient Journey (pj_id), Patient Journey Step (pj_step) and Patient Journey Phase (pj_phase) identifiers
# (These IDs are used for calculation purposes only.  They will not be published)

def pj(df):
    pj_df = (
        df
        .assign(**{
            'pj_id' : lambda x: (
                x.groupby([transform.patient_id, transform.pharmacy, transform.brand_col]).grouper.group_info[0]
            ),
            'pj_step' : lambda x: x.index,
            'pj_phase' : lambda x:(
                np.where(
                    ((x['pj_id'] == x['pj_id'].shift(1))
                     & (x[transform.status] == x[transform.status].shift(1))),
                    0,
                    1
                )
                .cumsum()
            )
        })
        .sort_values(
            by=[transform.patient_id, transform.pharmacy, transform.brand_col, transform.status_date, transform.status, transform.trans_id],
            ascending=[True, True, True, True, False, True])
        .reset_index(drop=True)
    )
    return pj_df

In [ ]:
# Assign True/False to "last_status_flag" to flag records that are the last in a given patient journey.
# Get the max(status_date) for the entire dataset.

def last_status(pj_df):
    last_status_df = (
        pj_df
        .assign(last_status_flag = lambda x: (
            np.where(
                x['pj_step'] == x.groupby(['pj_id'])['pj_step'].transform(max),
                True,
                False
            )
        ))
        .assign(max_status_date = lambda x:(
            x[transform.status_date].max()
        ))
    )
    return last_status_df

In [ ]:
# Filter the data to include only the last status for patient journeys that require a "pending too long" record to be inserted
# Manipulate the last status for that journey into the format of the inserted record (see Logic Diagram in description)

def pending_too_long(last_status_df):
    pending_too_long_df = (
        last_status_df
        .loc[lambda x: (
            ((x[transform.status] == transform.pending_status_code)
             &
             (x['last_status_flag'] == True)
             &
             ((x['max_status_date'] - x[transform.status_date]) / np.timedelta64(1,'D') > transform.ptl_threshold)
            ))
            ]
        .assign(**{
            transform.status : transform.cancel_status_code,
            transform.substatus : transform.pending_too_long_code,
            transform.hierarchy : transform.pending_too_long_code,
            transform.status_date : lambda x:(
                x[transform.status_date] + np.timedelta64(transform.ptl_threshold, 'D')),
            transform.unique_id : lambda x:('ic_' + x[transform.unique_id] + '_ptl')
        })
    )
    return pending_too_long_df

In [ ]:
# Run the pending_too_long enrichment

pending_too_long_df = (
    df
    .pipe(pj)
    .pipe(last_status)
    .pipe(pending_too_long)
)
    
pending_too_long_df.head()

In [ ]:
final_dataframe = (
    df
    .append(
        pending_too_long_df
        .drop(['pj_id', 'pj_step', 'pj_phase', 'last_status_flag', 'max_status_date'], axis=1)
    )
    .sort_values(
            by=[transform.patient_id, transform.pharmacy, transform.brand_col, transform.status_date, transform.status, transform.trans_id],
            ascending=[True, True, True, True, False, True])
    .reset_index(drop=True)
)

final_dataframe.head()

### TEST TRANSFORM OUTPUT

In [ ]:
# TEST 1: Create a "test" dataframe with expected results. Test the following scenarios:
#    1) Last status not pending and > 60 days from max(status_date) -> Leave as is
#    2) Last status is pending but not > 60 days from max(status_date) -> Leave as is
#    3) Last status is pending and > 60 days from max(status_date) -> Insert "PENDING TOO LONG" record
#    4) Multiple statuses reported on last day, some of which are pending, others not. -> Leave as is

test_data = ([
    ['01', 1, 0, 'ACTIVE', 'READY'],
    ['02', 1, 1, transform.pending_status_code, 'OTHER'],
    ['03', 1, 2, 'ACTIVE', 'READY'],
    ['04', 2, 0, transform.cancel_status_code, 'OTHER'],
    ['05', 2, 1, transform.pending_status_code, 'OTHER'],
    ['06', 3, 1, transform.cancel_status_code, 'OTHER'],
    ['07', 3, 1, transform.pending_status_code, 'OTHER'],
    ['08', 3, 1, transform.cancel_status_code, 'OTHER'],
    ['09', 4, 1, transform.pending_status_code, 'OTHER'],
    ['10', 4, 99, transform.pending_status_code, 'OTHER'],
    ['11', 5, 100, 'ACTIVE', 'SHIPMENT']
])

test_df = (
    pd.DataFrame(test_data, columns = [transform.unique_id, transform.patient_id, transform.status_date, transform.status, transform.substatus])
    .assign(**{
        transform.pharmacy : 'ABC',
        transform.brand_col : 'A',
        transform.status_date : lambda x: (
            pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x[transform.status_date], unit='d')),
        transform.hierarchy : None,
        transform.trans_id : 1
    })
    .sort_values(
            by=[transform.patient_id, transform.pharmacy, transform.brand_col, transform.status_date, transform.status, transform.trans_id],
            ascending=[True, True, True, True, False, True])
    .reset_index(drop=True)
)

test_df

In [ ]:
# Apply transform to test dataframe

test_output = (
    test_df
    .pipe(pj)
    .pipe(last_status)
    .pipe(pending_too_long)
)

final_dataframe_test = (
    test_df
    .append(
        test_output
        .drop(['pj_id', 'pj_step', 'pj_phase', 'last_status_flag', 'max_status_date'], axis=1),
        sort=False
    )
    .sort_values(
            by=[transform.patient_id, transform.pharmacy, transform.brand_col, transform.status_date, transform.status, transform.trans_id],
            ascending=[True, True, True, True, False, True])
    .reset_index(drop=True)
)

final_dataframe_test

In [ ]:
# Check that results match expectations

test1 = (
    (test_df.shape[0] == final_dataframe_test.shape[0] - 1) # Only 1 "pending too long" record created
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] != transform.pending_too_long_code].reset_index(drop=True).equals(test_df)) # Original data is not affected
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.status] == transform.cancel_status_code) # Pending too long status code applied
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.substatus] == transform.pending_too_long_code) # Pending too long substatus code applied
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.hierarchy] == transform.pending_too_long_code) # Pending too long hierarchy applied
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.unique_id] == 'ic_05_ptl') # Modified trans_id is correct
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.patient_id] == 2) # patient_id is correct
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.status_date] == pd.to_datetime('2019-03-03')) # status_date is correct
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.pharmacy] == 'ABC') # pharmacy is correct
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.brand_col] == 'A') # brand is correct
    &
    (final_dataframe_test.loc[lambda x: x[transform.substatus] == transform.pending_too_long_code, transform.trans_id] == 1) # brand is correct
).all()

test1

In [ ]:
# TEST 2: Check that final dataframe, minus the inserted records, has the same number of rows as the input dataframe

test2 = (final_dataframe.loc[lambda x: x[transform.substatus] != transform.pending_too_long_code].shape[0] == df.shape[0])

test2

In [ ]:
# TEST 3: Check that all "PENDING TOO LONG" records are preceded by "PENDING" statuses.

test3 = (
    final_dataframe
    .loc[lambda x:
         (x.shift(-1)[transform.status] == transform.pending_too_long_code)
        ]
    [transform.status]
    == transform.pending_status_code
).all()

test3

In [ ]:
# FINAL TEST: Did all 3 tests pass?

test1 & test2 & test3

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()